In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pandas as pd
from langdetect import detect
import re

In [ ]:
cService = webdriver.ChromeService(executable_path="..\chromedriver.exe")

In [2]:
driver = webdriver.Chrome(service=cService)

The chromedriver version (123.0.6312.122) detected in PATH at d:\DS\DS108\FINAL PROJECT Demo\Duy\crawler\chromedriver.exe might not be compatible with the detected chrome version (124.0.6367.119); currently, chromedriver 124.0.6367.155 is recommended for chrome 124.*, so it is advised to delete the driver in PATH and retry


In [3]:
url="https://www.itjobs.com.vn/en"

In [4]:
try:
    driver.get(url)
except Exception as e:
    print(e)

In [5]:
available_jobs = int(re.sub(r'\W','',driver.find_element(By.CLASS_NAME,"jp_job_heading").find_element(By.TAG_NAME,"span").text))
available_jobs

2046

In [6]:
try:
    while(driver.find_element(By.ID,"btnShowMoreJob")):
        driver.find_element(By.ID,"btnShowMoreJob").click()
        time.sleep(0.5)
        if(len(driver.find_elements(By.CLASS_NAME,"jp_job_post_link"))>=available_jobs):
            break
except:
    pass
jobs = driver.find_elements(By.CLASS_NAME,"jp_job_post_link")

In [7]:
def handle_text_reqirement(text):
    try:
        text=re.findall(r'\w.*\w', text)[0]
    except:
        return None
    text=text.strip()
    return text

In [8]:
def filter_en_requirement(reqs):
    reqs=[req for req in reqs if req is not None]
    for req in reqs:
        if(detect(req)=="vi"):
            return None
    for i in range(len(reqs)):
        if("benefit") in reqs[i].lower():
            return reqs[:i]
    return reqs

In [9]:
value=[]
columns=["Title","Company","Requirement","Salary","technical_skills","location","worktime","img_link","url"]

In [10]:
opitons=Options()
prefs = {
  "translate_whitelists": {"vi":"en"},
  "translate":{"enabled":"true"}
}
opitons.add_argument("--lang=en")
opitons.add_experimental_option("prefs",prefs)
newDriver = webdriver.Chrome()

for job in jobs:

    job_url=job.get_attribute("href")

    try:
        newDriver.get(job_url)
        time.sleep(0.2)
    except Exception as e:
        pass

    try:
        job_title = newDriver.find_element(By.CLASS_NAME,"jp_latest_job").find_element(By.TAG_NAME,"h3").text
        comany_name = newDriver.find_element(By.CLASS_NAME,"jp_latest_job").find_element(By.TAG_NAME,"p").text

        job_req=newDriver.find_element(By.CLASS_NAME,"job-requirement-section").find_element(By.CLASS_NAME,"jp_overview_wrapper").text

        final_job_req=filter_en_requirement(list(map(handle_text_reqirement,job_req.split("\n"))))

        salary=newDriver.find_element(By.CLASS_NAME,"jp_latest_job").find_element(By.CLASS_NAME,"marging-left-4").text

        technical_skills=newDriver.find_element(By.CLASS_NAME,"tech-skills-detail-page").text.split()

        job_location=newDriver.find_element(By.CLASS_NAME,"jp_latest_job").find_element(By.CLASS_NAME,"color-black").text

        job_worktime=newDriver.find_element(By.CLASS_NAME,"jp_latest_job").find_element(By.CLASS_NAME,"width-100").text
        
        img_link=newDriver.find_element(By.CLASS_NAME,"company-logo").find_element(By.TAG_NAME,"img").get_attribute("src")

        record=[job_title,comany_name,str(final_job_req),salary,str(technical_skills),job_location,job_worktime,img_link,job_url]
        
        value.append(record)
    except Exception as e:
        pass


The chromedriver version (123.0.6312.122) detected in PATH at d:\DS\DS108\FINAL PROJECT Demo\Duy\crawler\chromedriver.exe might not be compatible with the detected chrome version (124.0.6367.119); currently, chromedriver 124.0.6367.155 is recommended for chrome 124.*, so it is advised to delete the driver in PATH and retry


In [11]:
df=pd.DataFrame(value,columns=columns)
df

,Title,Company,Requirement,Salary,technical_skills,location,worktime,img_link,url
0,Senior C# Fullstack Developer,TOG recruitment,"['Must have', 'Above 8 years of experience in ...",Up to 70000000VND,"['C#', '.NET', 'MS', 'Azure', 'ASP.NET', 'ASP....","Quận 3, TP Hồ Chí Minh",Full Time,https://www.itjobs.com.vn/upload/Employer/TOG-...,https://www.itjobs.com.vn/en/job/81250/senior-...
1,Senior Java Backend Developer,Dr.JOY,"['Must have', 'Java (8 or more) - 5 years of e...",Up to 2000,"['Java', 'Spring', 'Spring', 'Boot', 'OOP', 'D...","Cầu Giấy, Hà Nội",Full Time,https://www.itjobs.com.vn/upload/Employer/Dr.J...,https://www.itjobs.com.vn/en/job/81724/senior-...
2,Salesforce Marketing Cloud Consultant,Clever Age,['Graduated from an engineering school or univ...,Negotiable,"['Salesforce', 'JavaScript']","Quận 3, TP Hồ Chí Minh",Full Time,https://www.itjobs.com.vn/upload/Employer/Clev...,https://www.itjobs.com.vn/en/job/81725/salesfo...
3,Frontend Developer,Fastboy Marketing,"['Must have', 'Proficient in React stack (Reac...",Negotiable,"['JavaScript', 'ReactJS', 'HTML5', 'CSS3', 'SC...","Tan Phu, TP Hồ Chí Minh",Full Time,https://www.itjobs.com.vn/upload/Employer/Fast...,https://www.itjobs.com.vn/en/job/81726/fronten...
4,Senior Product Owner,One Mount Group,['Minimum of 5 years of product management exp...,Negotiable,"['Product', 'Development', 'Jira']","Hoàn Kiếm, Hà Nội",Full Time,https://www.itjobs.com.vn/upload/Employer/One-...,https://www.itjobs.com.vn/en/job/81727/senior-...
...,...,...,...,...,...,...,...,...,...
2165,Embedded ECU Security Engineer,Bosch Global Software Technologies (BGSV),"['Your skills and experience', 'Mandatory requ...",Negotiable,"['C', 'Embedded', 'Algorithm', 'AutoSAR']","Tân Bình, TP Hồ Chí Minh",Full Time,https://www.itjobs.com.vn/upload/Employer/Bosc...,https://www.itjobs.com.vn/en/job/79668/embedde...
2166,PC Application Developer,Bosch Global Software Technologies (BGSV),"['Your skills and experience', 'Bachelor / Mas...",Negotiable,"['.NET', 'C#', 'iOS', 'Jira', 'Visual', 'Studi...","Tân Bình, TP Hồ Chí Minh",Full Time,https://www.itjobs.com.vn/upload/Employer/Bosc...,https://www.itjobs.com.vn/en/job/79670/pc-appl...
2167,Automation Test Software Engineer,Bosch Global Software Technologies (BGSV),"['Your skills and experience', '1-2 years work...",Negotiable,"['Python', 'Tester', 'Automation', 'Testing', ...","Tân Bình, TP Hồ Chí Minh",Full Time,https://www.itjobs.com.vn/upload/Employer/Bosc...,https://www.itjobs.com.vn/en/job/79666/automat...
2168,Senior AWS DevOps Engineer,ELCA Vietnam,"['Your skills and experience', 'We expect from...",Negotiable,"['DevOps', 'AWS', 'CI/CD', 'Java', '.NET', 'Ma...","Bình Thạnh, TP Hồ Chí Minh",Full Time,https://www.itjobs.com.vn/upload/Employer/ELCA...,https://www.itjobs.com.vn/en/job/79661/senior-...


In [12]:
df.to_csv("itjobs.csv",index=False) #save to csv